<a href="https://colab.research.google.com/github/kuntald1/Face-Recognized-by-Static-Image/blob/main/Face_Recognized_by_Static_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install system dependencies (VERY IMPORTANT)

In [4]:
!apt-get update
!apt-get install -y cmake
!apt-get install -y build-essential


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,896 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,468 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,733 

Install Python dependencies

In [5]:
!pip install dlib


Install face_recognition

In [6]:
!pip install face_recognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=1cf89ebc182838a301517d28ae117f242277058b6f69e29b5ac564fd52ad15c7
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


Restart runtime (MANDATORY)

In [1]:
from fastapi import FastAPI, UploadFile, File
import numpy as np
import cv2
import face_recognition

app = FastAPI()


In [3]:
@app.post("/recognize")
async def recognize(file: UploadFile = File(...)):
    contents = await file.read()

    np_img = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(np_img, cv2.IMREAD_COLOR)

    if img is None:
        return {"success": False, "message": "Invalid image"}

    face_locations = face_recognition.face_locations(img)

    if not face_locations:
        return {"success": False, "message": "No face detected"}

    if len(face_locations) > 1:
        return {"success": False, "message": "Multiple faces detected"}

    face_encoding = face_recognition.face_encodings(
        img, face_locations
    )[0]

    if not KNOWN_FACE_ENCODINGS:
        return {"success": False, "message": "No registered faces"}

    distances = face_recognition.face_distance(
        KNOWN_FACE_ENCODINGS,
        face_encoding
    )

    min_distance = np.min(distances)
    best_match_index = np.argmin(distances)

    if min_distance > 0.6:
        return {"success": False, "message": "Face not recognized"}

    return {
        "success": True,
        "employeeId": KNOWN_FACE_IDS[best_match_index],
        "confidence": float(1 - min_distance)
    }


Run your FastAPI app


In [7]:
!uvicorn main:app --reload --host 0.0.0.0 --port 8000


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [8512] using WatchFiles
INFO:     Started server process [8514]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8514]
INFO:     Stopping reloader process [8512]


In [13]:
!ngrok authtoken 39YwnmRLVza9vVWoc9LRzhynZ4e_6JXynk1XoeK48uZL9Y8QF

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("Your FastAPI server is running at:", public_url)


Your FastAPI server is running at: NgrokTunnel: "https://nondisingenuous-jordynn-precorneal.ngrok-free.dev" -> "http://localhost:8000"


Complete Colab-friendly test workflow

Upload an image for registration

In [18]:
from google.colab import files
uploaded = files.upload()  # Upload one "face.jpg" for registration


Saving face.jpg to face.jpg


Register it in KNOWN_FACE_ENCODINGS

In [26]:
import face_recognition

KNOWN_FACE_ENCODINGS = []
KNOWN_FACE_IDS = []

# Load a known face at startup
try:
    image = face_recognition.load_image_file("face.jpg")  # make sure this file is in the same folder
    encoding = face_recognition.face_encodings(image)[0]
    KNOWN_FACE_ENCODINGS.append(encoding)
    KNOWN_FACE_IDS.append("EMP001")
    print("Known face loaded successfully.")
except Exception as e:
    print("Failed to load known face:", e)



Known face loaded successfully.


Test /recognize using the same or another face image:

In [31]:
from fastapi.testclient import TestClient

client = TestClient(app)

# Upload another image for recognition
uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, "rb") as f:
    response = client.post("/recognize", files={"file": (filename, f, "image/jpeg")})

print(response.json())


Saving face - Copy.jpg to face - Copy (1).jpg
{'success': True, 'employeeId': 'EMP001', 'confidence': 0.8752512842238208}


Start Uvicorn again:

In [34]:
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &


nohup: appending output to 'nohup.out'


Check your ngrok URL

In [36]:
from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("FastAPI server is running at:", public_url)



FastAPI server is running at: NgrokTunnel: "https://nondisingenuous-jordynn-precorneal.ngrok-free.dev" -> "http://localhost:8000"
